# 第一次综合练习

In [1]:
import numpy as np
import pandas as pd

## 【任务一】企业收入的多样性

【题目复述】收入熵反映了一个企业的产业收入多样性，定义式如下：

$$
I = - \sum_ip(x_i)\log(p(x_i))
$$

其中$p(x_i)$是企业该年产业$x_i$收入额占所有产业总收入的比重。

【任务】利用`company_data.csv`的数据计算收入熵$I$，并将计算结果作为新的一列添加至`company.csv`。

先读取待处理数据集，了解基本情况。

In [2]:
# 读取企业清单
company = pd.read_csv('./data/Task Special/Company.csv')
company.head()

,证券代码,日期
0,#000007,2014
1,#000403,2015
2,#000408,2016
3,#000408,2017
4,#000426,2015


In [3]:
# 读取企业详细信息
df = pd.read_csv('./data/Task Special/Company_data.csv')
df.head()

,证券代码,日期,收入类型,收入额
0,1,2008/12/31,1,1.084218e+10
1,1,2008/12/31,2,1.259789e+10
2,1,2008/12/31,3,1.451312e+10
3,1,2008/12/31,4,1.063843e+09
4,1,2008/12/31,5,8.513880e+08


In [4]:
company['证券代码'].nunique()

253

In [5]:
df['证券代码'].nunique()

3172

通过初步观察原始数据，可以得出基本思路如下：

1. 将`company_data.csv`中的数据按照**证券代码**和**日期**分组。
2. 定义一个函数，用来计算收入熵，这个函数返回一个标量。
3. （有可能不对）利用`agg`方法分组计算收入熵。或者考虑利用`transform`，然后取唯一值。
4. 将`company.csv`与操作后的表进行左连接。

需要解决的细节问题问题是：

1. 将`company_data.csv`的证券代码格式化。


In [6]:
# 将证券代码格式化
df['证券代码'] = ['#%06d' % x for x in df['证券代码']]

可以看到，第二张表记录了2008年至2016年的数据

In [7]:
df['日期'].value_counts()

2013/12/31    155994
2012/12/31    149876
2016/12/31    122270
2011/12/31    117346
2015/12/31    109262
2014/12/31     92848
2010/12/31     90877
2009/12/31     73401
2008/12/31     52148
Name: 日期, dtype: int64

将日期数据改为年份。

In [8]:
df['日期'] = [x.split('/')[0] for x in df['日期']]

In [9]:
df.head()

,证券代码,日期,收入类型,收入额
0,#000001,2008,1,1.084218e+10
1,#000001,2008,2,1.259789e+10
2,#000001,2008,3,1.451312e+10
3,#000001,2008,4,1.063843e+09
4,#000001,2008,5,8.513880e+08


先取出某一个公司，某一年的收入情况，进行初步尝试。

In [10]:
ex = df.loc[(df['证券代码']=='#000001') & (df['日期']=='2008')].reset_index()

生成新的一列，表示单项收入占比。

In [11]:
ex['比例'] = ex['收入额']/ex['收入额'].sum()

In [12]:
ex['比例']*np.log(ex['比例'])    

C:\Users\LWKM\Anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


0    -0.283522
1    -0.303478
2    -0.321419
3    -0.061724
4    -0.052001
5    -0.100929
6          NaN
7    -0.267157
8          NaN
9    -0.260948
10   -0.283090
11   -0.035319
12   -0.071894
13   -0.043679
Name: 比例, dtype: float64

这里出现了一个问题：若收入为**负或零**，则无法取对数。

In [13]:
df.loc[df['收入额']<=0]['证券代码'].nunique()

2904

In [14]:
df['证券代码'].nunique()

3172

实现任务时可以暂时排除存在负收入业务的公司。

In [15]:
# 生成有负收入企业的代码
al = set(df['证券代码'].unique())
neg = set(df.loc[df['收入额']<0]['证券代码'].unique())

In [16]:
# 取补集
pos = pd.DataFrame(list(al.difference(neg)),columns=['证券代码'])

In [17]:
# 保留各项收入均为正的企业
pos = pos.merge(df,on='证券代码',how='left')

In [18]:
pos.head()

,证券代码,日期,收入类型,收入额
0,#002451,2010,1,6.855732e+07
1,#002451,2010,2,2.667245e+08
2,#002451,2010,3,2.127605e+07
3,#002451,2010,4,3.565579e+08
4,#002451,2010,5,8.925571e+05


In [19]:
pos['比例'] = pos.groupby(['证券代码','日期'])['收入额'].transform(lambda x: x/x.sum())

In [20]:
entro = pos.groupby(['证券代码','日期'])['比例'].agg([('收入熵',lambda x: -(x*np.log(x)).sum())])

C:\Users\LWKM\Anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [21]:
entro = entro.reset_index()

In [22]:
entro['日期'] = entro['日期'].astype('int')

In [23]:
company = company.merge(entro,on=['证券代码','日期'],how='left')

**最终结果**如下：

In [24]:
company.head(10)

,证券代码,日期,收入熵
0,#000007,2014,3.070462
1,#000403,2015,2.790585
2,#000408,2016,2.818541
3,#000408,2017,NaN
4,#000426,2015,3.084266
5,#000426,2016,2.988900
6,#000426,2017,NaN
7,#000511,2012,3.123733
8,#000511,2013,3.121685
9,#000571,2014,2.858982


## 【任务二】组队学习信息表的变换

In [25]:
team = pd.read_excel('./data/Task Special/组队信息汇总表（Pandas）.xlsx')

In [26]:
team.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 24 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   所在群       21 non-null     object 
 1   队伍名称      21 non-null     object 
 2   队长编号      21 non-null     int64  
 3   队长_群昵称    21 non-null     object 
 4   队员1 编号    21 non-null     int64  
 5   队员_群昵称    21 non-null     object 
 6   队员2 编号    20 non-null     float64
 7   队员_群昵称.1  20 non-null     object 
 8   队员3 编号    18 non-null     float64
 9   队员_群昵称.2  18 non-null     object 
 10  队员4 编号    16 non-null     float64
 11  队员_群昵称.3  16 non-null     object 
 12  队员5 编号    14 non-null     float64
 13  队员_群昵称.4  14 non-null     object 
 14  队员6 编号    13 non-null     float64
 15  队员_群昵称.5  13 non-null     object 
 16  队员7 编号    10 non-null     float64
 17  队员_群昵称.6  10 non-null     object 
 18  队员8 编号    8 non-null      float64
 19  队员_群昵称.7  8 non-null      object 
 20  队员9 编号    4 non-null      float64


In [27]:
team.head()

,所在群,队伍名称,队长编号,队长_群昵称,队员1 编号,队员_群昵称,队员2 编号,队员_群昵称.1,队员3 编号,队员_群昵称.2,...,队员6 编号,队员_群昵称.5,队员7 编号,队员_群昵称.6,队员8 编号,队员_群昵称.7,队员9 编号,队员_群昵称.8,队员10编号,队员_群昵称.9
0,Pandas数据分析,你说的都对队,5,山枫叶纷飞,6,蔡,7.0,安慕希,8.0,信仰,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pandas数据分析,熊猫人,175,鱼呲呲,44,Heaven,37.0,吕青,50.0,余柳成荫,...,25.0,Never say never,55.0,K,120.0,Y.,28.0,X.Y.Q,151.0,swrong
2,Pandas数据分析,中国移不动,107,Y's,124,🥕,75.0,Vito,146.0,张小五,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Pandas数据分析,panda,11,太下真君,35,柚子,108.0,My,42.0,星星点灯,...,157.0,Zys,158.0,不器,102.0,嘉平佑染,NaN,NaN,NaN,NaN
4,Pandas数据分析,一路向北,13,黄元帅,15,化,16.0,未期,18.0,太陽光下,...,23.0,🚀,169.0,听风,189.0,Cappuccino,NaN,NaN,NaN,NaN


可以看出，本任务的主要目标是实现从宽表到长表的变形。

结合原表特征，应该采用`wide_to_long`方法。

本题技术难点：列名称并不理想，可能需要调整和格式化？

实在不行，可以考虑**暴力重命名**

In [28]:
cl = ['所在群','队伍名称','编号_队长','昵称_队长','编号_队员1','昵称_队员1',
      '编号_队员2','昵称_队员2','编号_队员3','昵称_队员3','编号_队员4','昵称_队员4',
      '编号_队员5','昵称_队员5','编号_队员6','昵称_队员6','编号_队员7','昵称_队员7',
      '编号_队员8','昵称_队员8','编号_队员9','昵称_队员9','编号_队员10','昵称_队员10']

In [29]:
team.columns = cl

In [30]:
team = pd.wide_to_long(team, stubnames = ['编号','昵称'], i = ['所在群','队伍名称'], j = 'member',sep = '_',suffix='.+')

In [31]:
team = team.reset_index()

In [32]:
# 保留非空记录
team = team.iloc[:,1:].loc[team['编号'].isnull()==False].reset_index(drop = True)

In [33]:
# 更改格式
team['编号'] = team['编号'].astype('int')

In [34]:
# 提取队长指标变量
team['是否队长'] = pd.get_dummies(team.member)['队长']

In [35]:
# 重排顺序
team = team.iloc[:,[4,0,3,2]]

In [36]:
team

,是否队长,队伍名称,昵称,编号
0,1,你说的都对队,山枫叶纷飞,5
1,0,你说的都对队,蔡,6
2,0,你说的都对队,安慕希,7
3,0,你说的都对队,信仰,8
4,0,你说的都对队,biubiu🙈🙈,20
...,...,...,...,...
141,0,七星联盟,Daisy,63
142,0,七星联盟,One Better,131
143,0,七星联盟,rain,112
144,1,应如是,思无邪,54


时间有限，上述解法十分笨拙，但似乎可以达到目标效果。

## 【任务三】美国大选投票情况

In [37]:
pop = pd.read_csv('./data/Task Special/county_population.csv')
vote = pd.read_csv('./data/Task Special/president_county_candidate.csv')

In [38]:
s = [x.strip('.') for x in pop['US County']]

In [39]:
state = [x.split(', ')[1] for x in s]
county = [x.split(', ')[0] for x in s]

In [40]:
pop.head()

,US County,Population
0,".Autauga County, Alabama",55869
1,".Baldwin County, Alabama",223234
2,".Barbour County, Alabama",24686
3,".Bibb County, Alabama",22394
4,".Blount County, Alabama",57826


In [41]:
pop['state'], pop['county'] = state, county

In [42]:
pop = pop.iloc[:,[2,3,1]]

In [43]:
pop.head()

,state,county,Population
0,Alabama,Autauga County,55869
1,Alabama,Baldwin County,223234
2,Alabama,Barbour County,24686
3,Alabama,Bibb County,22394
4,Alabama,Blount County,57826


In [44]:
vote.head()

,state,county,candidate,party,total_votes,won
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False
3,Delaware,Kent County,Howie Hawkins,GRN,420,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True


考虑将两表连接。

In [45]:
# 生成合并数据集
df = vote.merge(pop, on = ['state','county'], how = 'left')

In [46]:
df.tail()

,state,county,candidate,party,total_votes,won,Population
32172,Arizona,Maricopa County,Write-ins,WRI,1331,False,4485414.0
32173,Arizona,Mohave County,Donald Trump,REP,78535,True,212181.0
32174,Arizona,Mohave County,Joe Biden,DEM,24831,False,212181.0
32175,Arizona,Mohave County,Jo Jorgensen,LIB,1302,False,212181.0
32176,Arizona,Mohave County,Write-ins,WRI,37,False,212181.0


In [47]:
# 将州县设为行索引
df['total_county'] = df.groupby(['state','county'])['total_votes'].transform('sum')

In [48]:
vote_c = df.groupby(['state','county'])['total_votes'].agg('sum').reset_index()

为了计算方便，把总投票数合并到了人口表上。

In [49]:
pop = pop.merge(vote_c,on=['state','county'], how = 'inner')

In [50]:
pop['prop']=pop.total_votes / pop.Population

In [51]:
pop.loc[pop.prop > 0.5].count()

state          1435
county         1435
Population     1435
total_votes    1435
prop           1435
dtype: int64

**第一问结果**：有1435个县满足半数人口投票。

In [52]:
vote.head()

,state,county,candidate,party,total_votes,won
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False
3,Delaware,Kent County,Howie Hawkins,GRN,420,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True


In [53]:
outcome = vote.groupby(['state','candidate'])['total_votes'].agg('sum').reset_index()

In [54]:
outcome.head()

,state,candidate,total_votes
0,Alabama,Write-ins,7312
1,Alabama,Donald Trump,1441168
2,Alabama,Jo Jorgensen,25176
3,Alabama,Joe Biden,849648
4,Alaska,Write-ins,34210


第二问的关键是将长表转为宽表。

In [55]:
outcome = outcome.pivot(index='state',columns='candidate',values='total_votes')

In [56]:
us_total = outcome.sum()

In [57]:
us_total.name='us'

In [58]:
us_total = us_total.to_frame().T

In [59]:
outcome = outcome.append(us_total)

In [60]:
final = outcome.T.sort_values('us',ascending=False).T.iloc[:-1,:]

**第二问结果**如下：

In [61]:
final.head()

candidate,Joe Biden,Donald Trump,Jo Jorgensen,Howie Hawkins,Write-ins,Rocky De La Fuente,Gloria La Riva,Kanye West,Don Blankenship,Brock Pierce,...,Tom Hoefling,Ricki Sue King,Princess Jacob-Fambro,Blake Huber,Richard Duncan,Joseph Kishore,Jordan Scott,Gary Swing,Keith McCormic,Zachary Scalf
Alabama,849648.0,1441168.0,25176.0,NaN,7312.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alaska,153405.0,189892.0,8896.0,NaN,34210.0,318.0,NaN,NaN,1127.0,825.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arizona,1672143.0,1661686.0,51465.0,NaN,2032.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arkansas,423932.0,760647.0,13133.0,2980.0,NaN,1321.0,1336.0,4099.0,2108.0,2141.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
California,11109764.0,6005961.0,187885.0,81025.0,80.0,60155.0,51036.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
vote.head()

,state,county,candidate,party,total_votes,won
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False
3,Delaware,Kent County,Howie Hawkins,GRN,420,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True


In [63]:
# 分县计算总投票数
vote['total_county'] = vote.groupby(['state','county'])['total_votes'].transform('sum')

In [64]:
vote

,state,county,candidate,party,total_votes,won,total_county
0,Delaware,Kent County,Joe Biden,DEM,44552,True,87025
1,Delaware,Kent County,Donald Trump,REP,41009,False,87025
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False,87025
3,Delaware,Kent County,Howie Hawkins,GRN,420,False,87025
4,Delaware,New Castle County,Joe Biden,DEM,195034,True,287633
...,...,...,...,...,...,...,...
32172,Arizona,Maricopa County,Write-ins,WRI,1331,False,2069475
32173,Arizona,Mohave County,Donald Trump,REP,78535,True,104705
32174,Arizona,Mohave County,Joe Biden,DEM,24831,False,104705
32175,Arizona,Mohave County,Jo Jorgensen,LIB,1302,False,104705


In [65]:
# 仅保留拜登和川普
BT = vote.loc[vote.candidate.isin(['Joe Biden','Donald Trump'])].reset_index(drop=True)

In [66]:
BT['rate'] = BT.total_votes / BT.total_county

In [67]:
# 保留有效信息
BT = BT[['state','county','candidate','rate']]

In [68]:
# 长变宽
BT = BT.pivot(index = ['state','county'], columns = 'candidate', values = 'rate')

In [69]:
BT.head()

candidate               Donald Trump  Joe Biden
state   county                                 
Alabama Autauga County      0.714368   0.270184
        Baldwin County      0.761714   0.224090
        Barbour County      0.534512   0.457882
        Bibb County         0.784263   0.206983
        Blount County       0.895716   0.095694

In [70]:
BT['BT'] = BT['Joe Biden'] - BT['Donald Trump']

In [71]:
btc = BT.reset_index().groupby('state')['BT'].agg(lambda x: x.median()>0)

In [72]:
Biden_Sate = btc[btc].index.to_list()

**第三问**结果：Biden State如下

In [73]:
for x in Biden_Sate: print(x)

California
Connecticut
Delaware
District of Columbia
Hawaii
Massachusetts
New Jersey
Rhode Island
Vermont
